In [8]:
import sys
sys.path.append('/uio/kant/geo-metos-u7/franzihe/Documents/Thesis/Python')

import numpy as np
import netCDF4
import pandas as pd
import matplotlib.pyplot as plt

import createFolder as cF
np.warnings.filterwarnings('ignore')



In [9]:
m = ['11', '12', '01', '02', '03']
station = 'Haukeliseter'

### Retrieval data

In [10]:
txt_dir = '../../Data/Retrieval/'


In [11]:
def get_netCDF_variable(f, var_name, var, dim):
    v_0m = f.createVariable(varname=var_name, datatype=var.dtype, dimensions=dim, zlib=True)
    v_0m[:] = var[:]
    return(v_0m)

Haukeli = pd.read_fwf(txt_dir+txt_filename,header=1)

ret_diff = dict()
obs_f = dict()
obs_wind_dir_10m = dict()
obs_wind_speed_10m = dict()
obs_wind_dir = dict()
obs_wind_speed = dict()
obs_wind_dir_rolling = dict()
obs_wind_speed_rolling = dict()
obs_westerly_idx = dict()
obs_easterly_idx = dict()
obs_westerly = dict()
obs_easterly = dict()

for i in range(int(np.round((Haukeli.shape[0]-1)/int(1440/6+1)))):
    Date = int(Haukeli['Unnamed: 0'][i*int(1440/6+1)])
    precip_24 = (Haukeli['Unnamed: 1'][i*int(1440/6+1)])
    precip_24 = np.array([precip_24])
    
    column0 = Haukeli['Unnamed: 0'][i*int(1440/6+1)+1:(i+1)*int(1440/6+1)]
    column1 = Haukeli['Unnamed: 1'][i*int(1440/6+1)+1:(i+1)*int(1440/6+1)]
    column2 = Haukeli['Unnamed: 2'][i*int(1440/6+1)+1:(i+1)*int(1440/6+1)]
    column3 = Haukeli['Unnamed: 3'][i*int(1440/6+1)+1:(i+1)*int(1440/6+1)]
    column4 = Haukeli['Unnamed: 4'][i*int(1440/6+1)+1:(i+1)*int(1440/6+1)]
    column5 = Haukeli['Unnamed: 5'][i*int(1440/6+1)+1:(i+1)*int(1440/6+1)]

    column0 = column0.convert_objects(convert_numeric=True)
    column1 = column1.convert_objects(convert_numeric=True)
    column2 = column2.convert_objects(convert_numeric=True)
    column3 = column3.convert_objects(convert_numeric=True)
    column4 = column4.convert_objects(convert_numeric=True)
    column5 = column5.convert_objects(convert_numeric=True)

    ret_diff[Date] = pd.concat([column0, column1, column2, column3, column4, column5], axis =1)
    ret_diff[Date] = np.array(ret_diff[Date])

    ret_diff[Date] = ret_diff[Date].flatten()/ret_diff[Date].flatten().shape[0]
    

    

In [12]:
obs_f = dict()
obs_wind_dir_10m = dict()
obs_wind_speed_10m = dict()
obs_wind_dir = dict()
obs_wind_speed = dict()
obs_wind_dir_rolling = dict()
obs_wind_speed_rolling = dict()
obs_westerly_idx = dict()
obs_easterly_idx = dict()
obs_westerly = dict()
obs_easterly = dict()
obs_double_fence_acc_mean = dict()
obs_dofe_acc = dict()
obs_dofe_acc_rolling = dict()
obs_diff = dict()
obs_total_precip = dict()
obs_precip_east = dict()
obs_precip_west = dict()

In [13]:
m = ['11']

In [14]:
for month in m:
    if month == '11':
        t = np.arange(7,9)
    if month == '12' or month == '01' or month == '03':
        t = np.arange(1,32)
    if month == '02':
        t = np.arange(1,29)
    if month == '11' or month == '12':
        year = '2016'
    if month == '01' or month == '02' or month == '03':
        year = '2017'
    

    for day in t:
        
        if day < 10:
            day = '0%s' %day
        else:
            day = '%s' %day
        
        Date = int(year+month+day)
        obs_dirnc = '../../Data/Weathermast/Met_%s_%s.nc' %(Date,station)
        obs_f[Date] = netCDF4.Dataset(obs_dirnc, 'r')
        
        obs_double_fence_acc_mean[Date] = (np.nanmean(obs_f[Date].variables['double_fence_accum'][:], axis=1))
        obs_wind_dir_10m[Date]   = (obs_f[Date].variables['10m_wind_dir_mast1'][:])
        obs_wind_speed_10m[Date] = (obs_f[Date].variables['10m_wind_speed_mast1'][:])

        obs_f[Date].close()

        ## Create a rolling mean to smoothen out the data
        # attach the last 9 minutes from the previous day to calculate the rolling mean every 10 minutes
        if month != '11' and day == '01':
            if month == '12':
                previous_day = '30'
            elif month == '01' or month == '02':
                previous_day = '31'
            elif month == '03':
                previous_day = '28'
            if month == '01':
                previous_month = '12'
                previous_year = str(int(year)-1)
            elif month == '12':
                previous_month = str(int(month)-1)
                previous_year = year
            else:
                previous_month = '0%s' %(int(month)-1)
                previous_year = year
            
        elif int(day)-1 < 10:
            previous_day = '0%s' %(int(day)-1)
            previous_month = month
            previous_year = year
        else:
            previous_day = '%s' %(int(day)-1)
            previous_month = month
            previous_year = year

        if year == '2016' and month == '11' and day == '07':
            continue

        else:
            pp = (np.concatenate((obs_double_fence_acc_mean[int(previous_year+previous_month+previous_day)][-9:],
                                  obs_double_fence_acc_mean[Date][:])))
            obs_dofe_acc[Date] = (pp)
            wd = (np.concatenate((obs_wind_dir_10m[int(previous_year+previous_month+previous_day)][-9:], 
                                  obs_wind_dir_10m[Date][:])))
            obs_wind_dir[Date] = (wd)
            ws = (np.concatenate((obs_wind_speed_10m[int(previous_year+previous_month+previous_day)][-9:], 
                                  obs_wind_speed_10m[Date][:])))
            obs_wind_speed[Date] = (ws)
        # Calculate the moving average. That is, take
        # the first ten values, average them, 
        # then drop the first and add the eleventh, etc.
        obs_dofe_acc[Date] = pd.DataFrame(np.array(obs_dofe_acc[Date])[:])
        obs_dofe_acc_rolling[Date] = obs_dofe_acc[Date].rolling(window=10,
                                                                                              min_periods= 7,
                                                                                              axis=0).mean()
        obs_dofe_acc_rolling[Date] = obs_dofe_acc_rolling[Date].values
        obs_dofe_acc_rolling[Date] = np.array(obs_dofe_acc_rolling[Date])[9:,:]
        
        obs_wind_dir[Date] = pd.DataFrame(np.array(obs_wind_dir[Date])[:])
        obs_wind_dir_rolling[Date] = obs_wind_dir[Date].rolling(window=10,min_periods=7,axis=0).mean()
        obs_wind_dir_rolling[Date] = obs_wind_dir_rolling[Date].values
        obs_wind_dir_rolling[Date] = np.array(obs_wind_dir_rolling[Date])[9:,:]

        obs_wind_speed[Date] = pd.DataFrame(np.array(obs_wind_speed[Date])[:])
        obs_wind_speed_rolling[Date] = obs_wind_speed[Date].rolling(window=10,min_periods=7,axis=0).mean()
        obs_wind_speed_rolling[Date] = obs_wind_speed_rolling[Date].values
        obs_wind_speed_rolling[Date] = np.array(obs_wind_speed_rolling[Date])[9:,:]
        
        ## calculate the precipitation difference, since the minutelty data is accumulated from the beginning
        obs_diff[Date] = np.empty(shape=(obs_dofe_acc_rolling[Date].shape))#/60))
        obs_diff[Date][:] =  np.nan

        for i in np.arange(0,obs_dofe_acc_rolling[Date].shape[0]-1):#/60):
            obs_diff[Date][int(i),:] = obs_dofe_acc_rolling[Date][int(i)+1] - \
                                                      obs_dofe_acc_rolling[Date][int(i)]

        # Find wind regimes and associated precipitation pattern
        IDX = np.arange(obs_wind_dir_rolling[Date].shape[0])

        ## find all obs_westerly regimes (202.5 - 22.5 deg) 
        obs_westerly_idx[Date]  = IDX[np.logical_or(np.logical_and((obs_wind_dir_rolling[Date][:,0] >= 202.5), 
                                                                   (obs_wind_dir_rolling[Date][:,0] <= 360)),
                                                   (obs_wind_dir_rolling[Date][:,0] <22.5))]
        ## find all obs_easterly regimes (22.5 - 202.5 deg)
        obs_easterly_idx[Date]  = IDX[np.logical_and((obs_wind_dir_rolling[Date][:,0] >= 22.5),
                                                     (obs_wind_dir_rolling[Date][:,0] < 202.5))]

        ## find indices for all obs_westerly regimes 
        sub_idx = []

        for i in range(0,obs_westerly_idx[Date] .shape[0]-1):
            if obs_westerly_idx[Date] [i]+1 == obs_westerly_idx[Date] [i+1]:
                continue
            else:
                #print(i)
                sub_idx.append(i)
    
       
        ## find westerlies which are < 29 min and assign them to obs_easterly wind regime
        if len(obs_westerly_idx[Date]) == 0 and len(sub_idx) == 0:     # westerlies do not exist
            obs_westerly[Date] = []
            obs_easterly[Date] = obs_easterly_idx[Date]
        elif len(obs_westerly_idx[Date]) != 0 and len(sub_idx) == 0:   # westerlies exist only for one time period a day
            obs_westerly[Date] = []
            obs_easterly[Date] = obs_easterly_idx[Date]
            
            obs_westerly_duration = obs_westerly_idx[Date][-1] - obs_westerly_idx[Date][0]
            if obs_westerly_duration >= 28:
                obs_westerly[Date] = np.concatenate((obs_westerly[Date],obs_westerly_idx[Date][:]))
            elif obs_westerly_duration < 28:
                obs_easterly[Date] = np.concatenate((obs_easterly[Date],obs_westerly_idx[Date][:]))
        else:
            obs_westerly[Date] = []
            obs_easterly[Date] = obs_easterly_idx[Date]

            for i in np.arange(0,np.array(sub_idx).shape[0]+1):                # westerlies and easterlies interchange daily
                if i == 0:
                    obs_westerly_duration = (obs_westerly_idx[Date] [ sub_idx[0]] - \
                                         obs_westerly_idx[Date] [0])
                elif i == np.array(sub_idx).shape[0]:
                    obs_westerly_duration = (obs_westerly_idx[Date] [-1] - \
                                         obs_westerly_idx[Date] [ sub_idx[np.array(sub_idx).shape[0]-1]+1])
                else:
                    obs_westerly_duration = (obs_westerly_idx[Date] [ sub_idx[i]] - \
                                         obs_westerly_idx[Date] [ sub_idx[i-1]+1])
                if obs_westerly_duration >= 28:
                    if i == 0:
                        obs_westerly[Date] = np.concatenate((obs_westerly[Date], 
                                                             obs_westerly_idx[Date] [ 0: sub_idx[0]+1]))
                    elif i == np.array(sub_idx).shape[0]:
                        obs_westerly[Date] = np.concatenate((obs_westerly[Date], 
                                                             obs_westerly_idx[Date] [ sub_idx[np.array(sub_idx).shape[0]-1]+1 : ]))
                    else:
                        obs_westerly[Date] = np.concatenate((obs_westerly[Date], 
                                                             obs_westerly_idx[Date] [ sub_idx[i-1]+1 : sub_idx[i]+1]))
                elif obs_westerly_duration < 28:
                    if i == 0:
                        obs_easterly[Date] = np.concatenate((obs_easterly[Date], 
                                                             obs_westerly_idx[Date] [ 0: sub_idx[0]+1]))
                    elif i == np.array(sub_idx).shape[0]:
                        obs_easterly[Date] = np.concatenate((obs_easterly[Date],
                                                             obs_westerly_idx[Date] [ sub_idx[np.array(sub_idx).shape[0]-1]+1 : ]))
                    else:
                   # print(obs_westerly_duration)
                        obs_easterly[Date] = np.concatenate((obs_easterly[Date],
                                                             obs_westerly_idx[Date] [ sub_idx[i-1]+1 : sub_idx[i]+1]))
        
        if len(obs_westerly[Date]) == 0:    # only easterlies
            obs_easterly[Date]  = obs_easterly[Date].astype(int)
        elif len(obs_easterly[Date]) == 0:  # only westerlies
            obs_westerly[Date]  = obs_westerly[Date].astype(int)
        else:    # both easterlies and westerlies
            obs_easterly[Date]  = obs_easterly[Date].astype(int)
            obs_westerly[Date]  = obs_westerly[Date].astype(int)
            
        # assign sum values to days
        if len(obs_easterly[Date]) == 0 and len(obs_westerly[Date]) == 0:
            obs_total_precip[Date] = np.nan
            obs_precip_east[Date] =  np.nan
            obs_precip_west[Date] =  np.nan
        else:
            obs_total_precip[Date] = np.nansum(obs_diff[Date][obs_easterly_idx[Date],:]) + \
                                                np.nansum(obs_diff[Date][obs_westerly_idx[Date],:])
            obs_precip_east[Date] = np.nansum(obs_diff[Date][obs_easterly[Date],:])
            obs_precip_west[Date] = np.nansum(obs_diff[Date][obs_westerly[Date],:])
          
        
        
############################            
        # assign obs_diff values to wind regime

        if len(obs_westerly[Date]) != 0:
            # use pulsed file
            txt_filename = 'pulsed_minute_surface_acc_Haukeli.dat.txt'
            Haukeli = pd.read_fwf(txt_dir+txt_filename,header=1)
            idx = np.where(Haukeli['Unnamed: 0'] == Date)
            if np.array(idx).shape[1] == 0:
                print([Date], 'no retried values')
                ret_pulsed = np.empty(shape=(obs_wind_dir_rolling[Date].shape[0],))
                ret_pulsed[:] = np.nan
                ret_precip = np.empty(shape=(obs_wind_dir_rolling[Date].shape[0],))
                ret_precip[:] = np.nan
                ret_upslope = np.empty(shape=(obs_wind_dir_rolling[Date].shape[0],))
                ret_upslope[:] = np.nan
                ret_total_precip = np.empty(shape=(1,))
                ret_total_precip[:] = np.nan
                ret_precip_east = np.empty(shape=(1,))
                ret_precip_east[:] = np.nan
                ret_precip_west = np.empty(shape=(1,))
                ret_precip_west[:] = np.nan
                
            elif np.array(idx).shape[1] == 1:
                precip_24 = (Haukeli['Unnamed: 1'][idx[0][0]])
                precip_24 = np.array([precip_24])

                column0 = Haukeli['Unnamed: 0'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                column1 = Haukeli['Unnamed: 1'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                column2 = Haukeli['Unnamed: 2'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                column3 = Haukeli['Unnamed: 3'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                column4 = Haukeli['Unnamed: 4'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                column5 = Haukeli['Unnamed: 5'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]

                column0 = column0.convert_objects(convert_numeric=True)
                column1 = column1.convert_objects(convert_numeric=True)
                column2 = column2.convert_objects(convert_numeric=True)
                column3 = column3.convert_objects(convert_numeric=True)
                column4 = column4.convert_objects(convert_numeric=True)
                column5 = column5.convert_objects(convert_numeric=True)

                ret_pulsed = pd.concat([column0, column1, column2, column3, column4, column5], axis =1)
                ret_pulsed = np.array(ret_pulsed)

                ret_pulsed = ret_pulsed.flatten()/ret_pulsed.flatten().shape[0]

                # assign precip values to westerly wind regime, easterly wind regime will have nan
                ret_precip = np.empty(shape=ret_pulsed.shape)
                ret_precip[:] = np.nan
                ret_precip[obs_westerly[Date],] = ret_pulsed[obs_westerly[Date],]
                
                ret_upslope = np.empty(shape=ret_pulsed.shape)
                ret_upslope[:] = np.nan

                # then where it is nan assign easterly values
                if len(obs_easterly[Date]) != 0:
                    # use upslope file
                    txt_filename = 'upslope_minute_surface_acc_Haukeli.dat.txt'
                    Haukeli = pd.read_fwf(txt_dir+txt_filename,header=1)
                    idx = np.where(Haukeli['Unnamed: 0'] == Date)

                    column0 = Haukeli['Unnamed: 0'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                    column1 = Haukeli['Unnamed: 1'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                    column2 = Haukeli['Unnamed: 2'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                    column3 = Haukeli['Unnamed: 3'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                    column4 = Haukeli['Unnamed: 4'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                    column5 = Haukeli['Unnamed: 5'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]

                    column0 = column0.convert_objects(convert_numeric=True)
                    column1 = column1.convert_objects(convert_numeric=True)
                    column2 = column2.convert_objects(convert_numeric=True)
                    column3 = column3.convert_objects(convert_numeric=True)
                    column4 = column4.convert_objects(convert_numeric=True)
                    column5 = column5.convert_objects(convert_numeric=True)

                    ret_upslope = pd.concat([column0, column1, column2, column3, column4, column5], axis =1)
                    ret_upslope = np.array(ret_upslope)

                    ret_upslope = ret_upslope.flatten()/ret_upslope.flatten().shape[0]
                    #assing precip values to easterly wind regime
                    ret_precip[obs_easterly[Date],] = ret_upslope[obs_easterly[Date],]
                
 
        elif len(obs_westerly[Date]) == 0 and len(obs_easterly[Date]) != 0:
            # use upslope file
            txt_filename = 'upslope_minute_surface_acc_Haukeli.dat.txt'
            Haukeli = pd.read_fwf(txt_dir+txt_filename,header=1)
            idx = np.where(Haukeli['Unnamed: 0'] == Date)
            if np.array(idx).shape[1] == 0:
                print([Date], 'no retried values')
                ret_pulsed = np.empty(shape=(obs_wind_dir_rolling[Date].shape[0],))
                ret_pulsed[:] = np.nan
                ret_precip = np.empty(shape=(obs_wind_dir_rolling[Date].shape[0],))
                ret_precip[:] = np.nan
                ret_upslope = np.empty(shape=(obs_wind_dir_rolling[Date].shape[0],))
                ret_upslope[:] = np.nan
                ret_total_precip = np.empty(shape=(1,))
                ret_total_precip[:] = np.nan
                ret_precip_east = np.empty(shape=(1,))
                ret_precip_east[:] = np.nan
                ret_precip_west = np.empty(shape=(1,))
                ret_precip_west[:] = np.nan

            else:
                precip_24 = (Haukeli['Unnamed: 1'][idx[0][0]])
                precip_24 = np.array([precip_24])

                column0 = Haukeli['Unnamed: 0'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                column1 = Haukeli['Unnamed: 1'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                column2 = Haukeli['Unnamed: 2'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                column3 = Haukeli['Unnamed: 3'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                column4 = Haukeli['Unnamed: 4'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]
                column5 = Haukeli['Unnamed: 5'][idx[0][0]+1:int(idx[0][0]+1440/6+1)]

                column0 = column0.convert_objects(convert_numeric=True)
                column1 = column1.convert_objects(convert_numeric=True)
                column2 = column2.convert_objects(convert_numeric=True)
                column3 = column3.convert_objects(convert_numeric=True)
                column4 = column4.convert_objects(convert_numeric=True)
                column5 = column5.convert_objects(convert_numeric=True)

                ret_upslope = pd.concat([column0, column1, column2, column3, column4, column5], axis =1)
                ret_upslope = np.array(ret_upslope)

                ret_upslope = ret_upslope.flatten()/ret_upslope.flatten().shape[0]
                #assing precip values to easterly wind regime
                ret_precip = np.empty(shape=ret_upslope.shape)
                ret_precip[:] = np.nan
                ret_precip[obs_easterly[Date],] = ret_upslope[obs_easterly[Date],]

                ret_pulsed = np.empty(shape=ret_upslope.shape)
                ret_pulsed[:] = np.nan

In [19]:
obs_total_precip[Date]

-0.005333333332885104

In [20]:
obs_precip_east[Date]

-0.005333333332885104

In [21]:
obs_precip_west[Date]

0.0

In [24]:
ret_upslope

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
###############################################                 
        ### for minuetly data giving negative values use the eklima observations
        if obs_total_precip[Date] < 0:
            if obs_precip_east[Date] < 0 and obs_precip_west[Date] < 0:
                obs_wind_dir_rolling[Date] = eklima_wind_dir_10m[Date]
                obs_wind_speed_rolling[Date] = eklima_wind_speed_10m[Date]
                obs_westerly[Date] = eklima_westerly[Date]
                obs_easterly[Date] = eklima_easterly[Date]
            elif obs_precip_east[Date] < 0:    
                obs_wind_dir_rolling[Date] = eklima_wind_dir_10m[Date]
                obs_wind_speed_rolling[Date] = eklima_wind_speed_10m[Date]
                obs_westerly[Date] = eklima_westerly[Date]
                obs_easterly[Date] = eklima_easterly[Date]
            elif obs_precip_west[Date] < 0:
                obs_wind_dir_rolling[Date] = eklima_wind_dir_10m[Date]
                obs_wind_speed_rolling[Date] = eklima_wind_speed_10m[Date]
                obs_westerly[Date] = eklima_westerly[Date]
                obs_easterly[Date] = eklima_easterly[Date]
###############################        
        ### sum up the precipitation for 1hour to get the same resolution as the eklima data 
            for i in range(int(ret_precip.shape[0]/60)):
                ret_pulsed = np.sum(ret_pulsed[i*60:(i+1)*60])
                ret_upslope = np.sum(ret_upslope[i*60:(i+1)*60])
        ### assign for the wind regimes the hourly retrieval precipitation amount        
            if len(obs_westerly[Date]) != 0:  # use pulsed file
            # assign precip values to westerly wind regime, easterly wind regime will have nan
                ret_precip = np.empty(shape=ret_pulsed.shape)
                ret_precip[:] = np.nan
                ret_precip[obs_westerly[Date],] = ret_pulsed[obs_westerly[Date],]

                # then where it is nan assign easterly values
                if len(obs_easterly[Date]) != 0:   # use upslope file
                    #assing precip values to easterly wind regime
                    ret_precip[obs_easterly[Date],] = ret_upslope[obs_easterly[Date],]
            elif len(obs_westerly[Date]) == 0 and len(obs_easterly[Date]) != 0:    # use upslope file
            #assing precip values to easterly wind regime
                ret_precip = np.empty(shape=ret_upslope.shape)
                ret_precip[:] = np.nan
                ret_precip[obs_easterly[Date],] = ret_upslope[obs_easterly[Date],]
            
        elif obs_precip_east[Date] < 0 or \
             obs_precip_west[Date] < 0 or \
             np.isnan(obs_total_precip[Date]):
            obs_wind_dir_rolling[Date] = eklima_wind_dir_10m[Date]
            obs_wind_speed_rolling[Date] = eklima_wind_speed_10m[Date]
            obs_westerly[Date] = eklima_westerly[Date]
            obs_easterly[Date] = eklima_easterly[Date]
###############################        
        ### sum up the precipitation for 1hour to get the same resolution as the eklima data 
            for i in range(int(ret_precip.shape[0]/60)):
                ret_pulsed = np.sum(ret_pulsed[i*60:(i+1)*60])
                ret_upslope = np.sum(ret_upslope[i*60:(i+1)*60])
        ### assign for the wind regimes the hourly retrieval precipitation amount        
            if len(obs_westerly[Date]) != 0:  # use pulsed file
            # assign precip values to westerly wind regime, easterly wind regime will have nan
                ret_precip = np.empty(shape=ret_pulsed.shape)
                ret_precip[:] = np.nan
                ret_precip[obs_westerly[Date],] = ret_pulsed[obs_westerly[Date],]

                # then where it is nan assign easterly values
                if len(obs_easterly[Date]) != 0:   # use upslope file
                    #assing precip values to easterly wind regime
                    ret_precip[obs_easterly[Date],] = ret_upslope[obs_easterly[Date],]
            elif len(obs_westerly[Date]) == 0 and len(obs_easterly[Date]) != 0:    # use upslope file
            #assing precip values to easterly wind regime
                ret_precip = np.empty(shape=ret_upslope.shape)
                ret_precip[:] = np.nan
                ret_precip[obs_easterly[Date],] = ret_upslope[obs_easterly[Date],]
#        elif obs_precip_west[Date] < 0:
 #           obs_wind_dir_rolling[Date] = eklima_wind_dir_10m[Date]
  #          obs_wind_speed_rolling[Date] = eklima_wind_speed_10m[Date]
   #         obs_westerly[Date] = eklima_westerly[Date]
    #        obs_easterly[Date] = eklima_easterly[Date]
     #   elif np.isnan(obs_total_precip[Date]):
      #      obs_wind_dir_rolling[Date] = eklima_wind_dir_10m[Date]
       #     obs_wind_speed_rolling[Date] = eklima_wind_speed_10m[Date]
        #    obs_westerly[Date] = eklima_westerly[Date]
         #   obs_easterly[Date] = eklima_easterly[Date]

In [ ]:
Date

In [ ]:
len(obs_westerly[Date]) 

In [ ]:
len(obs_easterly[Date]) 

In [ ]:
        if len(obs_easterly[int(year+month+day)]) == 0 and len(obs_westerly[int(year+month+day)]) == 0:
                ret_total_precip = np.array([np.nan])
                ret_precip_east  =  np.array([np.nan])
                ret_precip_west  =  np.array([np.nan])
        else:
                ret_total_precip = np.array([np.nansum(ret_precip[:,])])
                ret_precip_east  = np.array([np.nansum(ret_precip[obs_easterly[Date],])])
                ret_precip_west  = np.array([np.nansum(ret_precip[obs_westerly[Date],])])
            
        print([Date], 'easterly:', np.round(ret_precip_east,2), 
                      'westerly:', np.round(ret_precip_west,2),
                      'total:', np.round(ret_total_precip,2))
        filename = 'precip_wind_%s.nc' %(Date)            
        cF.createFolder(txt_dir+station+'/combined/')

In [16]:
eklima_f = dict()
eklima_double_fence_acc = dict()
eklima_wind_dir_10m = dict()
eklima_wind_speed_10m = dict()
eklima_diff = dict()
eklima_westerly_idx = dict()
eklima_easterly_idx = dict()
eklima_westerly = dict()
eklima_easterly = dict()

for month in m:
    if month == '11':
        t = np.arange(8,31)
    if month == '12' or month == '01' or month == '03':
        t = np.arange(1,32)
    if month == '02':
        t = np.arange(1,29)
    if month == '11' or month == '12':
        year = '2016'
    if month == '01' or month == '02' or month == '03':
        year = '2017'
    

    for day in t:
        
        if day < 10:
            day = '0%s' %day
        else:
            day = '%s' %day

        eklima_dirnc = '../../Data/Eklima/%s/precip_wind_temp_%s%s%s.nc' %(station,year,month,day)
        eklima_f[int(year+month+day)] = netCDF4.Dataset(eklima_dirnc, 'r')

        eklima_double_fence_acc[int(year+month+day)] = (eklima_f[int(year+month+day)].variables['RR_1'][:])
        eklima_wind_dir_10m[int(year+month+day)]   = (eklima_f[int(year+month+day)].variables['DD'][:])
        eklima_wind_speed_10m[int(year+month+day)] = (eklima_f[int(year+month+day)].variables['FF'][:])
        
        eklima_f[int(year+month+day)].close
        

        ## calculate the precipitation difference, since the minutelty data is accumulated from the beginning
        eklima_diff[int(year+month+day)] = np.empty(shape=(eklima_double_fence_acc[int(year+month+day)].shape))#/60))
        eklima_diff[int(year+month+day)][:] =  np.nan

        eklima_diff[int(year+month+day)][:,] = eklima_double_fence_acc[int(year+month+day)]

        # Find wind regimes and associated precip amount
        IDX = np.arange(eklima_wind_dir_10m[int(year+month+day)].shape[0])

        ## find all eklima_westerly regimes (202.5 - 22.5 deg) 
        eklima_westerly_idx[int(year+month+day)]  = IDX[np.logical_or(np.logical_and((eklima_wind_dir_10m[int(year+month+day)][:,] >= 202.5),
                                                                                     (eklima_wind_dir_10m[int(year+month+day)][:,] <= 360)),
                                                                      (eklima_wind_dir_10m[int(year+month+day)][:,] <22.5))]

        ## find all eklima_easterly regimes (22.5 - 202.5 deg)
        eklima_easterly_idx[int(year+month+day)]  = IDX[np.logical_and((eklima_wind_dir_10m[int(year+month+day)][:,] >= 22.5),
                                                                       (eklima_wind_dir_10m[int(year+month+day)][:,] < 202.5))]

        eklima_westerly[int(year+month+day)] = eklima_westerly_idx[int(year+month+day)]
        eklima_easterly[int(year+month+day)] = eklima_easterly_idx[int(year+month+day)]

        if len(eklima_westerly[int(year+month+day)]) == 0:    # only easterlies
            eklima_easterly[int(year+month+day)]  = eklima_easterly[int(year+month+day)].astype(int)

        elif len(eklima_easterly[int(year+month+day)]) == 0:  # only westerlies
            eklima_westerly[int(year+month+day)]  = eklima_westerly[int(year+month+day)].astype(int)

        else:    # both easterlies and westerlies
            eklima_easterly[int(year+month+day)]  = eklima_easterly[int(year+month+day)].astype(int)
            eklima_westerly[int(year+month+day)]  = eklima_westerly[int(year+month+day)].astype(int)
            


In [ ]:
b = []
for i in range(int(ret_upslope.shape[0]/60)):
    print(np.sum(ret_precip[i*60:(i+1)*60]))
    a = np.sum(ret_precip[i*60:(i+1)*60])
    b.append(a)

In [ ]:
    

    ### write netCDF file
    fw = netCDF4.Dataset(txt_dir+station+'/combined/', 'w')
    ### create dimensions
    fw.createDimension('time', ret_diff.shape[0])
    fw.createDimension('24h_acc_sum', precip_24.shape[0])
    
    precip_w       = get_netCDF_variable(fw, 'precipitation', ret_diff, ('time',))
    total_precip_w = get_netCDF_variable(fw, '24h_precip_acc', precip_24, ('24h_acc_sum',) )
    
    fw.close()